# Quantitative MoM Function

#### This is the quantitative Momentum trade. This function will take as an input the dataframe of closing daily prices for the S&P 500 list. 

In [2]:
import pandas as pd
import numpy as np
import os as os
from datetime import datetime
import matplotlib.pyplot as plt
from pathlib import Path
#from backtesting import Backtest,Strategy
from datetime import datetime, timedelta

In [3]:
# Pick up the date we grabbed in our acquisition. Don't really need to rebalance often, so it doesn't make sense to pull via sdk and 
# waste the message count of whatever service we use. 

file_path = Path('Resources/sp500_latest.csv')

sp500_full = pd.read_csv(file_path,
                         parse_dates=True,
                         index_col = 'date',
                         infer_datetime_format=True)
sp500_full.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2020-04-02,72.29,10.06,88.41,244.93,75.13,83.89,143.28,79.44,156.27,303.96,...,79.15,40.40,37.47,18.48,63.40,66.05,92.71,174.24,25.39,116.03
2020-04-03,70.42,9.39,84.65,241.41,73.37,81.34,141.40,79.45,152.15,293.61,...,79.08,39.21,35.45,16.90,60.50,63.30,92.01,178.08,24.68,114.61
2020-04-06,74.36,9.50,95.81,262.47,75.73,86.29,150.26,82.73,166.05,319.13,...,84.65,40.47,37.73,18.71,65.19,70.50,97.94,191.85,26.93,126.79
2020-04-07,74.03,10.22,99.44,259.43,75.39,87.20,152.78,81.93,164.12,308.93,...,83.95,41.24,38.09,18.61,66.19,71.35,101.91,192.36,28.04,126.11
2020-04-08,76.69,11.33,102.52,266.07,78.56,88.33,156.27,84.95,171.73,317.18,...,84.50,43.85,39.58,19.26,69.09,75.37,111.28,196.65,29.44,127.25


In [4]:
# We need to determine the month and lookback to form the trade. Ultimately, the function will return a portfolio
# for a date range, dated for that month. We will pass the dataframe with that month to a backtesting to test the 
# value of the trade for every period. 

# Parameters for the function should include start date, end date, and then parse the months and form lookback portfolios
# over that date range. We will assume a monthly turnover if equities in the new portfolio were not in the previous
# portfolio. 

def generic_return(start,end,data):
    # update formatting for the dates
    start_date = start.strftime("%Y-%m-%d")
    end_date = end.strftime("%Y-%m-%d")
    
    data = data[start:end]
    
    #get the subset of data within the prescribed range
    #mask = (data['date'] > start_date) & (data['date'] <= end_date)
   # data=data.loc[mask].set_index('date')
    
    #calculate pct change during the period
    overall_returns = data.pct_change()
    
    #determine cumulative returns ending at the end of the period minus the last day
    cumulative_returns = (1 + overall_returns).cumprod() - 1
    
    #final total return for the period
    total_return = cumulative_returns.iloc[[-1]]
    
    #take the output and convert to a vertical list of tickers and their cumulative return with date
    return_list = total_return.stack()
    
    #reset the index 
    df = return_list.to_frame().reset_index()
    
    #set columns of 'symbol' and 'return'
    df.rename(columns ={'level_1':'symbol',0:"return"}, inplace = True)
    
    #drop the duplicate 'date' column now that index = 'date'
    df.drop(columns=['date'], inplace=True)
    
    #set the column of 'symbol' to the index
    df.set_index(df['symbol'], inplace = True)
    
    #drop the extra symbol column
    df.drop(columns=['symbol'], inplace=True)
    
    #break into deciles and add the decile column
    df['decile_rank']=pd.qcut(df['return'],10,labels=False)
    
    #get top decile
    top_decile = df.loc[df['decile_rank'] == 9]
    
#     print(top_decile)
#     print(f"Start date is {start_date} and End date is {end_date}")
    return top_decile



In [5]:
sp500_full.index = sp500_full.index.tz_localize(None)

In [6]:
# Create the parameters for start/end date

end_date = datetime.now()
start_date = end_date + timedelta(-365)

In [7]:
# Create the generic return portfolio

portfolio = generic_return(start_date,end_date,sp500_full)
portfolio

,return,decile_rank
symbol,,
ABMD,0.853992,9
ADSK,0.779642,9
ALGN,0.830670,9
AMP,0.577990,9
AMZN,0.605723,9
APA,1.786780,9
APTV,0.683703,9
BBY,0.652388,9
BWA,0.567774,9


In [8]:
# Take the tickers from the output from Generic momentum and the date range selected earlier to create the fip score over
# that range
def fip(start,end,top_decile,data):
    
    #reset the index on the Top Decile to get the tickers
    top_decile = top_decile.reset_index()
    ticker_top=list(top_decile.iloc[:,0])
    
    #filter the dataset upon the date range provided
    start_date = start.strftime("%Y-%m-%d")
    end_date = end.strftime("%Y-%m-%d")
    data = data[start:end]
    
    # filter the dataset upon the tickers provided
    # data = data[ticker_top]
    data = data[[s for s in top_decile.symbol if s in data.columns]]   
    
    # pass the ticker_top list and the date range to select the portion of data to run the Fip_Score
    
    # Calculating the pct_change for the fip calculation
    data = data.pct_change().dropna()
    
    #initialize an empty library to hold the calculated Fip Score
    fip_scores = []
    
    # Set up a for loop to set variables for the number of positive / negative / zero and number of data. 
    # Determine the cumulativ returns and set to a variable. 
    for column in data.columns:
        num_positive = len(data.loc[data[column] > 0])
        num_negative = len(data.loc[data[column] < 0])
        num_zero = len(data.loc[data[column] == 0])
        num_days = len(data)
        cum_returns = data[column].sum() 
        
        # Perform the Fip Score calculation
    
        fip_score = np.sign(cum_returns) * (num_negative / num_days - num_positive/num_days)
        
        # As the last step in the loop, append the fip scores to get the whole list
        fip_scores.append(fip_score)
    
    # create a pd.Series with the appended fip scores, set all to the variable FIP
    fip = pd.Series(fip_scores,index = data.columns)
    
    # set the name of the series to Fip
    fip.name = "fip"
    
    #create a top_decile variable to an empty data frame with the index set to symbol
    top_decile = top_decile.set_index('symbol')
    
    
    # This one is tough, I'll leave this in place as this is a more complicated approach than we've taken previously
    
    return top_decile.merge(fip,left_index = True, right_index = True).sort_values(['return','fip'],ascending=False)

In [9]:
# Call the Fip function, pass in paramaters of start date, end date, the portfolio, and the SP500 ticker list
# The output of this function is the top decile returns of momentum with a score of the FIP score. 
# Choose from this list and make your trading decisions!

fip_portfolio = fip(start_date,end_date,portfolio,sp500_full)

In [13]:
fip_portfolio.sort_values(by = ['fip'], inplace = True)
fip_portfolio

,return,decile_rank,fip
symbol,,,
EBAY,0.996232,9,-0.424242
SNPS,0.566110,9,-0.393939
AMZN,0.605723,9,-0.393939
ADSK,0.779642,9,-0.333333
WHR,0.666054,9,-0.303030
BBY,0.652388,9,-0.272727
PYPL,0.941510,9,-0.272727
DISH,0.686181,9,-0.272727
USD,0.709416,9,-0.272727


# Now that we have our top decile and FIP, let's consider seasonality. 
## How did this portfolio do with seasonality? 

Let's evaluate behaviors to buy/sell at the end of the year, end of the quarter and think about how it informs our strategy. 

In [16]:
eval_data = fip_portfolio.reset_index()
eval_data

,symbol,return,decile_rank,fip
0,EBAY,0.996232,9,-0.424242
1,SNPS,0.566110,9,-0.393939
2,AMZN,0.605723,9,-0.393939
3,ADSK,0.779642,9,-0.333333
4,WHR,0.666054,9,-0.303030
5,BBY,0.652388,9,-0.272727
6,PYPL,0.941510,9,-0.272727
7,DISH,0.686181,9,-0.272727
8,USD,0.709416,9,-0.272727
9,PAYC,0.760675,9,-0.272727


In [23]:
type(eval_data)

pandas.core.frame.DataFrame

In [20]:
# we need
top_decile_tickers

0     EBAY
1     SNPS
2     AMZN
3     ADSK
4      WHR
5      BBY
6     PYPL
7     DISH
8      USD
9     PAYC
10    ABMD
11    APTV
12    SWKS
13    HOLX
14     DHI
15    TSCO
16    DXCM
17     LOW
18    CTAS
19     LEN
20     APA
21      DD
22    CARR
23     FCX
24    EXPE
25    NVDA
26     NOW
27    IPGP
28     KMX
29     MRO
30     PHM
31    MKTX
32    FBHS
33     CMG
34     PFG
35    ALGN
36     BWA
37     AMP
38     HAL
39     DRI
40     KIM
41    MCHP
42     DFS
43    NCLH
44     GPS
45     RCL
46    VIAC
47     CCL
48     KSS
49     MPC
50     HBI
Name: symbol, dtype: object